In [7]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from numba import jit, prange
import numpy as np

point = Point(-73.3930, 5.047)
polygon = Polygon([[0, 0], [0, 1], [1, 1], [1, 0]])
print(polygon.contains(point))

False


In [8]:
def make_polygons(fep):
    # leer los archivos como lista de tuplas
    f = open("fep/"+fep, encoding='latin1').readlines()
    regions = {}
    poly = []

    for line in f:
        if line[0] != "L":
            l = line.split()
            # agregando puntos del poligono de la region
            poly.append([float(l[0]), float(l[1])])
        else:
            regions[line.strip("L").strip()] = np.asarray(poly)
            
    return regions

# leer los archivos como lista de tuplas
%time regions = make_polygons("BOYACA.fep")

CPU times: user 2.7 s, sys: 72 ms, total: 2.77 s
Wall time: 2.77 s


In [9]:
@jit(nopython=True)
def ray_tracing(point,poly):
    x,y = point
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

In [19]:
point2 = (0.5, 0.5)
point3 = (-73.18, 6.59)
polygon2 = np.array([[0, 0], [0, 1], [1, 1], [1, 0]])
print(ray_tracing(point2, polygon2))

True


In [15]:
def find_in_fep(point, regions):
    for region in regions:
        polygon = regions[region]
        is_in = ray_tracing(point, polygon)
        if is_in:
            print(region)
            
%time find_in_fep(point3, regions)

CPU times: user 92 ms, sys: 52 ms, total: 144 ms
Wall time: 142 ms


In [20]:
import os

def find_region(point):
    feps = [i for i in os.listdir("fep") if os.path.isfile(
                                       os.path.join("fep",i))]
    for fep in feps:
        regions = make_polygons(fep)
        find_in_fep(point, regions)

%time find_region(point3)

ConcepciÃ³n - Santander, Colombia
El Guacamayo - Santander, Colombia
Oiba - Santander, Colombia
ContrataciÃ³n - Santander, Colombia
Coromoro - Santander, Colombia
La Paz - Santander, Colombia
CharalÃ¡ - Santander, Colombia
Ocamonte - Santander, Colombia
Confines - Santander, Colombia
Palmas del Socorro - Santander, Colombia
Macaravita - Santander, Colombia
Capitanejo - Santander, Colombia
San Miguel - Santander, Colombia
Cabrera - Santander, Colombia
Hato - Santander, Colombia
Mogotes - Santander, Colombia
Zapatoca - Santander, Colombia
Albania - Santander, Colombia
GÃ¼epsa - Santander, Colombia
ChipatÃ¡ - Santander, Colombia
San Benito - Santander, Colombia
Suaita - Santander, Colombia
Aguada - Santander, Colombia
Bolivar - Santander, Colombia
Guadalupe - Santander, Colombia
FloriÃ¡n - Santander, Colombia
JesÃºs MarÃ­a - Santander, Colombia
Puente Nacional - Santander, Colombia
Barbosa - Santander, Colombia
La Belleza - Santander, Colombia
Piedecuesta - Santander, Colombia
San Vicente